##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Классификация изображений

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом уроке показано как классифицировать изображения цветов. Мы создадим классификатор изображений с использованием модели `keras.Sequential` и загрузим данные с помощью `preprocessing.image_dataset_from_directory`. 

Вы получите практический опыт работы со следующими концепциями:
* Эффективная загрузка набора данных с диска.
* Выявление переобучения и применение методов для его уменьшения, включая увеличение количества данных и Dropout.

В этом руководстве описываются основные шаги построения процесса машинного обучения:

1. Изучение и понимание данных
2. Создание конвейера для входных данных
3. Построение модели
4. Обучение модели
5. Проверка модели
6. Улучшение модели и повторение процесса

## Импорт TensorFlow и других библиотек

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
print(tf.__version__)

## Загрузка и просмотр датасета

В этом уроке используется датасет из примерно 3700 фотографий цветов. Датасет состоит из 5-ти директорий, по одной на каждый класс(цветок)ю

```
flower_photo/
  daisy/
  dandelion/
  roses/
  sunflowers/
  tulips/
```

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

После загрузки вам доступна копия датасета. Всего 3,670 изображений:

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

Посмотрим изображения из директории roses:

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
PIL.Image.open(str(roses[1]))

И из tulips:

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[0]))

In [ ]:
PIL.Image.open(str(tulips[1]))

# Загрузка с диска с использованием keras.preprocessing

Давайте загрузим изображения с диска с помощью утилиты [image_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory). Этот помощник создаст `tf.data.Dataset` всего-лишь несколькими строчками кода. Или, если вам нравится, вы можете создать загрузчик изображений с нуля, используя [load images](https://www.tensorflow.org/tutorials/load_data/images).

## Создание датасета

Определим некторые параметры для загрузки:

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

Считается хорошей практикой использовать проверочный сет при обучении модели. Мы будем использовать 80% изображений для тренировки, и 20% для проверки.

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Вы можете посмотреть классы датасета в аттрибуте `class_names`. В данном случае они соответствуют именам директорий и отсортированы в алфавитном порядке.

In [ ]:
class_names = train_ds.class_names
print(class_names)

## Визуализация данных

Первые 9 изображений из тренировочного датасета.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Вы будет отправлять эти датасеты в метод `model.fit` вашей модели. Вы также можете пройти методом for по датасету и посмотреть на пакет, возвращаемый датасетом:

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

`image_batch` это тензор размерности `(32, 180, 180, 3)`. В данном примере: `32` - размер пакета, `180х180` - размер изображения, `3` - количество цветовых каналов(RGB). `label_batch` - тензор размерности `(32,)`, в данном примере содержит 32 метки для изображений. 

Вы моежет вызвать метод `.numpy()` на `image_batch` и `labels_batch` тензорах, чтобы преобразовать их в `numpy.ndarray`.


## Конфигурирование датасета для улучшения производительности

Обязательно используйте буферизованную предварительную выборку, чтобы мы могли читать данные с диска без блокировки ввода/вывода. Вот два важных метода, которые вы должны использовать при загрузке данных.

`Dataset.cache()` сохраняет изображения в памяти после их загрузки с диска в течение первой эпохи. Это гарантирует, что загрузка данных не станет узким местом при обучении вашей модели. Если ваш набор данных слишком большой и не помещается в память, вы также можете использовать этот метод для создания дискового кеша..

`Dataset.prefetch()` перекрывает предварительную обработку данных и выполнение модели во время обучения за счет предварительной выборки.

Заинтересованные читатели могут узнать больше об обоих методах, а также о том, как кэшировать данные на диск, в [руководстве по производительности данных] (https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Нормализация данных

The RGB channel values are in the `[0, 255]` range. This is not ideal for a neural network; in general you should seek to make your input values small. Here, we will standardize values to be in the `[0, 1]` by using a Rescaling layer.

Значения канала RGB находятся в диапазоне `[0, 255]`. Это не идеально для нейронной сети, вы должны стремиться к тому, чтобы ваши входные значения были небольшими. Далее мы нормализуем значения, чтобы они находились в диапазоне `[0, 1]`, используя слой Rescaling.

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

Примечание. Утилиты и слои Keras Preprocesing, описанные в этом разделе, в настоящее время являются экспериментальными и могут измениться.

Существует два способа использовать слой Rescaling. Вы можете применить его к набору данных, вызвав метод map:

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Обратите внимание, что значения пикселей теперь находятся в `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

Или вы можете включить слой непосредственно в модель, что упростит развертывание. Дальше мы воспользуемся вторым подходом.

Примечание: ранее мы изменяли размер изображений используя аргумент `image_size` метода `image_dataset_from_directory`. Если вы хотите включить изменения размера в свою модель, вы можете использовать слой [Resizing](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Resizing).

# Создание модели

The model consists of three convolution blocks with a max pool layer in each of them. There's a fully connected layer with 128 units on top of it that is activated by a `relu` activation function. This model has not been tuned for high accuracy, the goal of this tutorial is to show a standard approach. 

Модель состоит из трех сверточных слоев Conv2D с MaxPooling2D слоем в каждом из них. Далее следует полносвязанный слой со 128 входными параметрами, который активируется функцией активации `relu`. Эта модель не настроена для обеспечения высокой точности, цель этого руководства - показать стандартный подход.

In [ ]:
num_classes = 5

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

## Компиляция модели

Для этого урока выберем оптимайзер `optimizers.Adam` и функцию потерь `losses.SparseCategoricalCrossentropy`. Для просмотра тренировочной и проверочной точности на каждой эпохе, передадим аргумент `metrics`.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Структура модели

Для просмотра всех слоев модели используйте метод `summary`:

In [ ]:
model.summary()

## Тренировка модели

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Визуализация результатов

Построим графики потерь и точности для тренировочных и проверочных данных

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Как вы можете видеть на графиках, точность проверки значительно ниже точности обучения, и модель достигла всего-лишь около 70% точности на проверочных данных.

Давайте посмотрим, что пошло не так, и попробуем повысить общую производительность модели.

## Переобучение(Overfitting)

На графиках выше точность обучения линейно увеличивается со временем, тогда как точность проверки в процессе обучения составляет 60 - 70%. Разница между точностью обучения и точностью проверки - признак [переобучения](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit).

Когда модель обучается на небольшом количестве данных, она начинает учиться на шумах или нежелательных деталях обучающих примеров. До такой степени, что это отрицательно влияет на точность модели на новых примерах. Это явление известно как переобучение. Это значит, что модели будет сложно обобщить новый набор данных.

Есть несколько способов борьбы с переобучением в тренировочном процессе. В этом руководстве вы будете использовать *увеличение данных* и добавление *Dropout* в свою модель.

## Увеличение данных(Data augmentation)

Переобучение обычно происходит при небольшом количестве обучающих примеров. [Увеличение данных](https://www.tensorflow.org/tutorials/images/data_augmentation) использует подход, основанный на создании дополнительных обучающих данных из существующих примеров, путем использования случайных преобразований, которые дают правдоподобные изображения. Это дает возможность модели потренироваться на большем количестве данных и лучше обобщить.

Для аугментации данных мы будем использовать [Keras Preprocessing Layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/?version=nightly). Слои предварительной обработки(Keras Preprocessing Layers) могут быть включены в вашу модель, как и другие слои, и запускаться на графическом процессоре.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

Давайте визуализируем несколько примеров, созданных с помощью Keras Preprocessing Layers:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

Мы используем аугментацию данных и заново обучим модель

## Исключение из обучения(Dropout)

Еще один метод уменьшения переобучения - добавить в модель [Dropout](https://developers.google.com/machine-learning/glossary#dropout_regularization) как форму *регуляризации*.

Когда вы применяете Dropout к слою, он случайным образом исключает из слоя(путем установки активации на ноль) некоторое количество выходных единиц во время процесса обучения. Dropout принимает дробное число в качестве входного значения, например 0.1, 0.2, 0.4 и т.д. Это означает случайное исключение 10%, 20% или 40% выходных единиц из применяемого слоя.

Давайте создадим новую нейронную сеть, используя `layers.Dropout`, а затем обучим ее, используя созданные с помощью аугментации изображения.

In [ ]:
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

## Компиляция и тренировка модели

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Визуализация результатов

После применения аугментации данных и слоя `Dropout` переобучение уменьшилось, а точность тренировки и точность проверки стали гораздо ближе.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Предсказание на новых данных

В конце давайте проверим как наша модель классифицирует изображение, которого не было ни в тренировочных, ни в проверочных данных.

Примечание: Аугментация данных и `Dropout` неактивны во время получения предсказаний(метод predict).

In [ ]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = keras.preprocessing.image.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)